In [49]:
# This notebook explores some of the math used for generating SVM signals. It's taken from ST app note AN2154 on applying SVM to an 8-bit microcontroller.

In [50]:
import math
import numpy as np

In [51]:
# Sampling frequency (Hz). I think this is the overflow rate of the timer, or maybe the time it 
# takes to complete a full up/down count cycle?
fs = 10000.0

# Elapsed time per counter tick
timer_base = 10000000.0  # 10 Mhz
tp = 1/timer_base

# Timer max value before count down. Divide by two because we have to make two trips before the PWM cycle completes.
tmax = np.uint32((timer_base / fs) / 2)
tmax_mod = np.uint32(tmax + 1)

# Modulation Index (max sqrt(3)/2, or 0.866)
mi = 0.5

# Electrical Angle (Radians). Must not exceed PI/3 because the modulation scheme is only valid for 
# controls within a single 60 degree sector. Once we exceed that, it's moved on to a new switching 
# pattern and a new PWM time allocation.
alpha = math.radians(10.0)

In [52]:
# Equation 17: PWM Period
Ts = 1.0/(2.0*fs)
print(f"Sampling rate is {Ts*1e6} uS")

Sampling rate is 50.0 uS


In [53]:
# Equations 10, 11, 12: Compute the ratio of time spent in Vector A (starting vector), Vector B (final vector), and the Null vector.
ta = (2/math.sqrt(3))*Ts*mi*math.sin((math.pi/3.0)-alpha)
tb = (2/math.sqrt(3))*Ts*mi*math.sin(alpha)
tnull = Ts-ta-tb
print(f"Time in vectors from total of {Ts*1e6} uS:\n\tTa: {ta*1e6}\n\tTb: {tb*1e6}\n\tTnull: {tnull*1e6}")

Time in vectors from total of 50.0 uS:
	Ta: 22.11379827229795
	Tb: 5.01279110601451
	Tnull: 22.873410621687544


In [54]:
# Equations 14, 15, 16: Compute the capture compare register values
ta_ticks = np.uint32(ta/tp)
tb_ticks = np.uint32(tb/tp)
print(f"Ta_ticks: {ta_ticks}\nTb_ticks: {tb_ticks}")

Ta_ticks: 221
Tb_ticks: 50


In [59]:
inv_ta_tb = (~ta_ticks + tb_ticks) / 2

phase_u_ticks = (~(ta_ticks + tb_ticks) / 2) % tmax_mod
phase_v_ticks = (~(inv_ta_tb) + ta_ticks) % tmax_mod
phase_w_ticks = (~inv_ta_tb) % tmax_mod

print(phase_u_ticks)
print(phase_v_ticks)
print(phase_w_ticks)


TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''